#обучение

In [1]:
!pip install datasets tokenizers transformers evaluate

In [2]:
from datasets import Dataset, Features, Value, ClassLabel, Sequence

true_labels = ["O", "B-MRK", "I-MRK"]

# Функция для чтения данных из файла
def read_ner_file(file_path, labels):
    tokens, tags = [], []
    current_tokens, current_tags = [], []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if current_tokens:
                    tokens.append(current_tokens)
                    tags.append(current_tags)
                    current_tokens, current_tags = [], []
            else:
                parts = line.split()
                if len(parts) == 2:
                    current_tokens.append(parts[0])
                    current_tags.append(parts[1])

                    if parts[1] not in labels:
                      labels.append(parts[1])
                      labels.append(f'I-{parts[1][2:]}')

    return {"tokens": tokens, "ner_tags": tags}

# Создаем Dataset
data = read_ner_file("ner_tags_train.txt", true_labels)
dataset = Dataset.from_dict({
    "tokens": data["tokens"],
    "ner_tags": data["ner_tags"]
})

data1 = read_ner_file("ner_tags_test.txt", true_labels)
dataset1 = Dataset.from_dict({
    "tokens": data1["tokens"],
    "ner_tags": data1["ner_tags"]
})

In [3]:
print(true_labels)

['O', 'B-MRK', 'I-MRK']


In [4]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification

In [5]:
from transformers import AutoTokenizer

model_name = "ai-forever/ruRoberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True, num_labels=len(true_labels))

# Создаем соответствие меток и их ID
label2id = {label: i for i, label in enumerate(true_labels)}
id2label = {i: label for i, label in enumerate(true_labels)}

# Обновляем датасет с ID меток
def align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512, padding="max_length")
    # tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, tags in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Игнорируем специальные токены
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[tags[word_idx]])
            else:
                label_ids.append(label2id[tags[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(align_labels, batched=True)
tokenized_dataset1 = dataset1.map(align_labels, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2660 [00:00<?, ? examples/s]

Map:   0%|          | 0/1414 [00:00<?, ? examples/s]

In [6]:

for sample in tokenized_dataset1:
  if len(sample["input_ids"]) != len(sample["labels"]):
    print(len(sample["input_ids"]), len(sample["labels"]))





In [7]:
from transformers import TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(true_labels),
    id2label=id2label,
    label2id=label2id
)

args = TrainingArguments(
    "argmark-ner",
    learning_rate = 2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy = "epoch",
    fp16=False
    # save_strategy="epoch",
    # load_best_model_at_end=True
)

data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
!pip install seqeval

In [9]:
import evaluate
import numpy as np
metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
  pred_logits, labels = eval_preds
  pred_logits = np.argmax(pred_logits, axis = 2)
  predictions = [
      [true_labels[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] for prediction, label in zip(pred_logits, labels)
  ]
  labels1 = [
      [true_labels[l] for (eval_preds, l) in zip(prediction, label) if l != -100] for prediction, label in zip(pred_logits, labels)
  ]
  results = metric.compute(predictions=predictions, references=labels1)
  return {
      "precision": results['overall_precision'],
      "recall": results['overall_recall'],
      "f1":results['overall_f1'],
      "accuracy": results['overall_accuracy']

  }

In [10]:
#143a6efb55865fb3a8c75c47ff9e7eaea1062e2e
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset1,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-10-fdea3d001cd0>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: a-araslanova (a-araslanova-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.115800,0.069011,0.854856,0.796610,0.824706,0.978908
2,0.055800,0.052860,0.856164,0.889831,0.872673,0.984264


TrainOutput(global_step=2660, training_loss=0.09403506185775412, metrics={'train_runtime': 3138.4135, 'train_samples_per_second': 1.695, 'train_steps_per_second': 0.848, 'total_flos': 4940737949736960.0, 'train_loss': 0.09403506185775412, 'epoch': 2.0})

In [11]:
model.config.label2id = label2id
model.config.id2label = id2label


model.save_pretrained("argmrkner_model")
tokenizer.save_pretrained("argmrkner_tknzr")


('argmrkner_tknzr/tokenizer_config.json',
 'argmrkner_tknzr/special_tokens_map.json',
 'argmrkner_tknzr/vocab.json',
 'argmrkner_tknzr/merges.txt',
 'argmrkner_tknzr/added_tokens.json',
 'argmrkner_tknzr/tokenizer.json')

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
